In [1]:
import csv
import numpy as np
import pandas as pd
import time

In [2]:
# load metro & mhi info

In [3]:
metro_dict = {} # dictionary: RegionName -> metro_id
metro_df_mat = []

In [4]:
def load_metro_mhi(filename):
    # begin time
    print('processing ' + filename)
    t0 = time.time()
    # prepare
    attributes = ['id','time_stamp','income']
    original_df = pd.read_csv(filename, encoding="ISO-8859-1")
    raw_df = original_df.drop(columns=['SizeRank'])
    mhi_df_mat = []
    def metro_mhi(series):
        # series: [metro_id, RegionName, timestamps...]
        name_code = [ x.strip() for x in series[1].split(',') ]
        metro_df_mat.append([ series[0], series[1], name_code[-1] ])
        metro_dict[series[1]] = series[0]
        for j in range(2,len(series)):
            if pd.isnull(series[j]):
                income = None
            else:
                income = int(series[j])
            mhi_df_mat.append([ series[0], series.index[j], income ])
    # apply to df
    raw_df.apply(metro_mhi, axis=1)
    temp_df = pd.DataFrame(mhi_df_mat, columns=attributes)
    # end time
    print('time used to process ' + str(filename) + ': ' + str( int(time.time()-t0) ) + 's')
    # return
    return temp_df

In [5]:
mhi_df = load_metro_mhi('rawdata/Affordability_Income_2018Q4.csv')
mhi_df.to_csv('processed_data/final_mhi.csv',index=False)

processing rawdata/Affordability_Income_2018Q4.csv
time used to process rawdata/Affordability_Income_2018Q4.csv: 4s


In [6]:
# load SalesCount

In [7]:
def load_sales(filename,property_type):
    # begin time
    print('processing ' + filename)
    t0 = time.time()
    # prepare
    attributes = ['metro_id', 'property_type', 'time_stamp', 'num_sales' ]
    original_df = pd.read_csv(filename, encoding="ISO-8859-1")
    raw_df = original_df.drop(columns=['SizeRank'])
    df_mat = []
    def unwind(series):
        # series: metro_id, RegionName, timestamp
        # for metro info
        metro_dict[series[1]] = series[0]
        name_code = [ x.strip() for x in series[1].split(',') ]
        metro_df_mat.append([ series[0], series[1], name_code[-1] ])
        # for SalesCount info
        metro_id = series[0]
        for j in range(2, len(series)):
            if pd.isnull(series[j]):
                value = None
            else:
                value = int(series[j])
            df_mat.append([ int(metro_id), property_type, series.index[j], value ])
    # apply map
    raw_df.apply(unwind, axis=1)
    temp_df = pd.DataFrame(df_mat, columns=attributes)
    # end time
    print('time used to process ' + str(filename) + ': ' + str( int(time.time()-t0) ) + 's')
    # return
    return temp_df

In [8]:
sales_df = load_sales('rawdata/Sale_Counts_Seas_Adj_Msa.csv','allhomes')
sales_df.to_csv('processed_data/final_SalesCount.csv',index=False)

processing rawdata/Sale_Counts_Seas_Adj_Msa.csv
time used to process rawdata/Sale_Counts_Seas_Adj_Msa.csv: 2s


In [9]:
# load price-to-rent ratio & sale-to-list ratio

In [10]:
# for price-to-rent ratio
def load_ptr(filename, attributes):
    # begin time
    print('processing ' + filename)
    t0 = time.time()
    # prepare
    attributes = ['metro_id', 'time_stamp', 'ptr_ratio' ]
    original_df = pd.read_csv(filename, encoding="ISO-8859-1")
    raw_df = original_df.drop(columns=['SizeRank'])
    df_mat = []
    def unwind(series):
        # series: metro_id, RegionName, timestamp
        # for metro info
        metro_dict[series[1]] = series[0]
        name_code = [ x.strip() for x in series[1].split(',') ]
        metro_df_mat.append([ series[0], series[1], name_code[-1] ])
        # for prt ratio info
        metro_id = series[0]
        for j in range(2, len(series)):
            if pd.isnull(series[j]):
                value = None
            else:
                value = series[j]
            df_mat.append([ int(metro_id), series.index[j], value ])
    # apply map
    raw_df.apply(unwind, axis=1)
    temp_df = pd.DataFrame(df_mat, columns=attributes)
    # end time
    print('time used to process ' + str(filename) + ': ' + str( int(time.time()-t0) ) + 's')
    # return
    return temp_df

# for sale-to-list ratio
def load_stl(filename, attributes):
    # begin time
    print('processing ' + filename)
    t0 = time.time()
    # prepare
    attributes = ['metro_id', 'time_stamp', 'stl_ratio' ]
    original_df = pd.read_csv(filename, encoding="ISO-8859-1")
    raw_df = original_df.drop(columns=['SizeRank','RegionType'])
    df_mat = []
    def unwind(series):
        # series: metro_id, RegionName, timestamp
        # for metro info
        metro_dict[series[1]] = series[0]
        name_code = [ x.strip() for x in series[1].split(',') ]
        metro_df_mat.append([ series[0], series[1], name_code[-1] ])
        # for prt ratio info
        metro_id = series[0]
        for j in range(2, len(series)):
            if pd.isnull(series[j]):
                value = None
            else:
                value = series[j]
            df_mat.append([ int(metro_id), series.index[j], value ])
    # apply map
    raw_df.apply(unwind, axis=1)
    temp_df = pd.DataFrame(df_mat, columns=attributes)
    # end time
    print('time used to process ' + str(filename) + ': ' + str( int(time.time()-t0) ) + 's')
    # return
    return temp_df

In [11]:
ptr_ratio_df = load_ptr('rawdata/Metro_PriceToRentRatio_AllHomes.csv', [])
ptr_ratio_df.to_csv('processed_data/final_ptr_ratio.csv',index=False)

stl_ratio_df = load_stl('rawdata/SaleToListRatio_Msa.csv', [])
stl_ratio_df.to_csv('processed_data/final_stl_ratio.csv',index=False)

processing rawdata/Metro_PriceToRentRatio_AllHomes.csv
time used to process rawdata/Metro_PriceToRentRatio_AllHomes.csv: 2s
processing rawdata/SaleToListRatio_Msa.csv
time used to process rawdata/SaleToListRatio_Msa.csv: 1s


In [12]:
# load HomeValue

In [13]:
def load_to_df(filename,property_type,room_num):
    # begin time
    print('processing ' + filename)
    t0 = time.time()
    # prepare
    attributes = ['metro_id','property_type','time_stamp','room_num','price']
    original_df = pd.read_csv(filename, encoding="ISO-8859-1")
    raw_df = original_df.drop(columns=['SizeRank'])
    df_mat = []
    def unwind(series):
        # series: metro_id, RegionName, timestamp
        # for metro info
        metro_dict[series[1]] = series[0]
        name_code = [ x.strip() for x in series[1].split(',') ]
        metro_df_mat.append([ series[0], series[1], name_code[-1] ])
        # for homevalue info
        metro_id = series[0]
        for j in range(2, len(series)):
            if pd.isnull(series[j]):
                price = None
            else:
                price = int(series[j])
            if pd.isnull(room_num):
                num = None
            else:
                num = int(room_num)
            df_mat.append([ int(metro_id), property_type, series.index[j], num, price ])
    # apply map
    raw_df.apply(unwind, axis=1)
    temp_df = pd.DataFrame(df_mat, columns=attributes)
    # end time
    print('time used to process ' + str(filename) + ': ' + str( int(time.time()-t0) ) + 's')
    # return
    return temp_df

In [14]:
homevalue_condo_df = load_to_df('rawdata/HomeValues-Condo.csv','condo',np.nan)
homevalue_condo_df.to_csv('processed_data/processed_homevalue_condo.csv',index=False)

processing rawdata/HomeValues-Condo.csv
time used to process rawdata/HomeValues-Condo.csv: 2s


In [15]:
homevalue_sf_df = load_to_df('rawdata/HomeValues-SingleFamily.csv','singlefamily',np.nan)
homevalue_sf_df.to_csv('processed_data/processed_homevalue_singlefamily.csv',index=False)

processing rawdata/HomeValues-SingleFamily.csv
time used to process rawdata/HomeValues-SingleFamily.csv: 5s


In [16]:
homevalue_1bedroom_df = load_to_df('rawdata/HomeValues-1bedroom.csv','1bedroom',1)
homevalue_1bedroom_df.to_csv('processed_data/processed_homevalue_1bedroom.csv',index=False)

processing rawdata/HomeValues-1bedroom.csv
time used to process rawdata/HomeValues-1bedroom.csv: 3s


In [17]:
homevalue_2bedroom_df = load_to_df('rawdata/HomeValues-2bedroom.csv','2bedroom',2)
homevalue_2bedroom_df.to_csv('processed_data/processed_homevalue_2bedroom.csv',index=False)

processing rawdata/HomeValues-2bedroom.csv
time used to process rawdata/HomeValues-2bedroom.csv: 4s


In [18]:
homevalue_3bedroom_df = load_to_df('rawdata/HomeValues-3bedroom.csv','3bedroom',3)
homevalue_3bedroom_df.to_csv('processed_data/processed_homevalue_3bedroom.csv',index=False)

processing rawdata/HomeValues-3bedroom.csv
time used to process rawdata/HomeValues-3bedroom.csv: 4s


In [19]:
homevalue_4bedroom_df = load_to_df('rawdata/HomeValues-4bedroom.csv','4bedroom',4)
homevalue_4bedroom_df.to_csv('processed_data/processed_homevalue_4bedroom.csv',index=False)

processing rawdata/HomeValues-4bedroom.csv
time used to process rawdata/HomeValues-4bedroom.csv: 4s


In [20]:
# save metro info

In [21]:
metro_df = pd.DataFrame(metro_df_mat, columns=['ID','name','state_code']).drop_duplicates(subset=['ID']).sort_values(by=['ID'])
metro_df.to_csv('processed_data/metro.csv',index=False)

In [22]:
# load RentalPrice

In [23]:
def load_rental(filename,property_type,room_num,metro_dict):
    # begin time
    print('processing ' + filename)
    t0 = time.time()
    # prepare
    attributes = ['metro_id','property_type','time_stamp','room_num','price']
    original_df = pd.read_csv(filename)
    raw_df = original_df.drop(columns=['SizeRank'])
    df_mat = []
    def unwind_rental(series):
        if series[0] in metro_dict:
            metro_id = metro_dict[series[0]]
            for j in range(1, len(series)):
                if pd.isnull(series[j]):
                    price = None
                else:
                    price = int(series[j])
                if pd.isnull(room_num):
                    num = None
                else:
                    num = int(room_num)
                df_mat.append([ metro_id, property_type, series.index[j], num, price ])
    # apply map
    raw_df.apply(unwind_rental, axis=1)
    temp_df = pd.DataFrame(df_mat, columns=attributes)
    # end time
    print('time used to process ' + str(filename) + ': ' + str( int(time.time()-t0) ) + 's')
    # return
    return temp_df

In [24]:
rentalprice_condo_df = load_rental('rawdata/Metro_MedianRentalPrice_CondoCoop.csv','condo',np.nan,metro_dict)
rentalprice_condo_df.to_csv('processed_data/processed_Metro_MedianRentalPrice_CondoCoop.csv',index=False)

processing rawdata/Metro_MedianRentalPrice_CondoCoop.csv
time used to process rawdata/Metro_MedianRentalPrice_CondoCoop.csv: 0s


In [25]:
rentalprice_sf_df = load_rental('rawdata/Metro_MedianRentalPrice_Sfr.csv','singlefamily',np.nan,metro_dict)
rentalprice_sf_df.to_csv('processed_data/processed_Metro_MedianRentalPrice_Sfr.csv',index=False)

processing rawdata/Metro_MedianRentalPrice_Sfr.csv
time used to process rawdata/Metro_MedianRentalPrice_Sfr.csv: 0s


In [26]:
rentalprice_1bedroom_df = load_rental('rawdata/Metro_MedianRentalPrice_1Bedroom.csv','1bedroom',1,metro_dict)
rentalprice_1bedroom_df.to_csv('processed_data/processed_Metro_MedianRentalPrice_1Bedroom.csv',index=False)

processing rawdata/Metro_MedianRentalPrice_1Bedroom.csv
time used to process rawdata/Metro_MedianRentalPrice_1Bedroom.csv: 0s


In [27]:
rentalprice_2bedroom_df = load_rental('rawdata/Metro_MedianRentalPrice_2Bedroom.csv','2bedroom',2,metro_dict)
rentalprice_2bedroom_df.to_csv('processed_data/processed_Metro_MedianRentalPrice_2Bedroom.csv',index=False)

processing rawdata/Metro_MedianRentalPrice_2Bedroom.csv
time used to process rawdata/Metro_MedianRentalPrice_2Bedroom.csv: 0s


In [28]:
rentalprice_3bedroom_df = load_rental('rawdata/Metro_MedianRentalPrice_3Bedroom.csv','3bedroom',3,metro_dict)
rentalprice_3bedroom_df.to_csv('processed_data/processed_Metro_MedianRentalPrice_3Bedroom.csv',index=False)

processing rawdata/Metro_MedianRentalPrice_3Bedroom.csv
time used to process rawdata/Metro_MedianRentalPrice_3Bedroom.csv: 0s


In [29]:
rentalprice_4bedroom_df = load_rental('rawdata/Metro_MedianRentalPrice_4Bedroom.csv','4bedroom',4,metro_dict)
rentalprice_4bedroom_df.to_csv('processed_data/processed_Metro_MedianRentalPrice_4Bedroom.csv',index=False)

processing rawdata/Metro_MedianRentalPrice_4Bedroom.csv
time used to process rawdata/Metro_MedianRentalPrice_4Bedroom.csv: 0s


In [30]:
# integrate data

In [31]:
homevalue_df = pd.concat([homevalue_condo_df,
                          homevalue_sf_df,
                          homevalue_1bedroom_df,
                          homevalue_2bedroom_df,
                          homevalue_3bedroom_df,
                          homevalue_4bedroom_df], ignore_index=True)
homevalue_df.to_csv('processed_data/final_homevalue.csv',index=False)

In [32]:
rentalprice_df = pd.concat([rentalprice_condo_df,
                            rentalprice_sf_df,
                            rentalprice_1bedroom_df,
                            rentalprice_2bedroom_df,
                            rentalprice_3bedroom_df,
                            rentalprice_4bedroom_df], ignore_index=True)
rentalprice_df.to_csv('processed_data/final_rentalprice.csv',index=False)